https://www.youtube.com/watch?v=gKspuNylqrs&list=LL

In [105]:
'''

This code aims to plot gravity & magnetic anomaly and perform filtering

by:
arif.darmawan@riflab.com
9 February 2023

'''

'\n\nThis code aims to plot gravity & magnetic anomaly and perform filtering\n\nby:\narif.darmawan@riflab.com\n9 February 2023\n\n'

In [106]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import pandas as pd
import numpy as np
import rasterio
from scipy.interpolate import griddata
from rasterio.transform import Affine
from rasterio.crs import CRS
import interpies

# # cartopy
# import cartopy
# import cartopy.crs as ccrs
# import cartopy.feature as cfeature

In [107]:
def set_map(ax, frame=True, dxdy=2000):
    ax.invert_yaxis()
    ax.set_box_aspect(1)
    ax.ticklabel_format(useOffset=False, style='plain')

    ax.xaxis.label.set_fontsize(10)
    ax.yaxis.label.set_fontsize(10)
    # ax.set_xlabel('Easting')
    # ax.set_ylabel('Northing')
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.tick_params(axis='x', labelsize=8, rotation=0)
    ax.tick_params(axis='y', labelsize=8, rotation=0)

    # start, end = ax.get_xlim()
    # start = (round(start, -3))
    # ax.xaxis.set_ticks(np.arange(start, end, dxdy))
    
    # start, end = ax.get_ylim()
    # start = (round(start, -3))
    # ax.yaxis.set_ticks(np.arange(start, end, dxdy))

    # ax.set_xlim(107.370, 107.470)
    # ax.set_ylim(-7.210, -7.110)

    if frame == False:
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.spines['left'].set_visible(False)
        ax.get_xaxis().set_ticks([])
        ax.get_yaxis().set_ticks([])
        ax.set_xlabel('')
        ax.set_ylabel('')

In [108]:
def mask_area(grid_grav, gridx, gridy):
    mask = (gridx > 107.43021) & (gridy > -7.16952)
    grid_grav[mask] = np.nan

    mask = (gridx > 107.44743) & (gridy > -7.17395)
    grid_grav[mask] = np.nan

    mask = (gridx < 107.38757) & (gridy < -7.14715)
    grid_grav[mask] = np.nan

    mask = (gridx > 107.42013) & (gridy > -7.14517)
    grid_grav[mask] = np.nan

    return grid_grav


In [109]:
def main(i, df, file_output, X='Longitude', Y='Latitude', epsg=4326, name='Title'):
    points = list(zip(df[X], df[Y]))
    values = df[i].values
    # print(points, i)
    
    min_x = df[X].min()
    max_x = df[X].max()
    min_y = df[Y].min()
    max_y = df[Y].max()
    # print(min_x, max_x, min_y, max_y)

    res1 = (max_x - min_x)/1000
    res2 = (max_y - min_y)/1000

    xrange = np.arange(min_x, max_x + res1, res1)
    yrange = np.arange(min_y, max_y + res2, res2)
    gridx, gridy = np.meshgrid(xrange, yrange)
    
    grid_grav = griddata(points, values, (gridx, gridy), method='cubic', fill_value=np.nan, rescale=True)

    # grid_grav = mask_area(grid_grav, gridx, gridy)
    
    # fig, ax = plt.subplots(figsize=(10, 10))
    # # im = ax.imshow(grid_grav, extent=(min_x, max_x, min_y, max_y), origin='lower')
    # ax.plot(df[X], df[Y], color='black', linestyle='None', marker='+', markersize=5, alpha=1)
    # set_map(ax, frame=True, dxdy=0.01)

    rasterCRS = CRS.from_epsg(epsg) #32748
    transform = Affine.translation(gridx[0][0]-res1/2, gridy[0][0]-res2/2)*Affine.scale(res1, res2)

    rasterfile = rasterio.open(
                                file_output, 'w', 
                                driver='GTiff',
                                dtype=grid_grav.dtype, 
                                count=1, 
                                width=grid_grav.shape[1], 
                                height=grid_grav.shape[0],
                                transform=transform,
                                crs=rasterCRS
                            )

    rasterfile.write(grid_grav, indexes=1)
    rasterfile.close()

    grid1 = interpies.open(file_output)

    ax1 = grid1.smooth(method='SG', deg=1, win=21, doEdges=True, sigma=1)
    ax1.save(file_output)
    # ax1.add_feature(cfeature.COASTLINE)
    ax1 = ax1.show(
                    cmap_norm='equalize', 
                    contours=True,
                    cb_contours=True, 
                    azdeg=315, altdeg=45, 
                    title=name,
                    # zf=1000,
                    figsize=(10, 10),
                    cmap='geosoft'
                    )
    set_map(ax1, frame=True, dxdy=0.01)
    


    # ax1.plot(df['X'], df['Y'], color='black', linestyle='None', marker='+', markersize=2, alpha=0.5)
    
    # # [50, 100, 200, 300, 400, 500, 1000,2000,3000,4000,5000]

    # z = [500]
    # for i in z:
    #     ax1 = grid1.hp_filter_uc(z=i)
    #     # ax1 = ax1.dxdy(
    #     #                 method='SG', 
    #     #                 deg=11, 
    #     #                 win=21, 
    #     #                 doEdges=True, 
    #     #                 fs_tap=5
    #     #                 )
    #     ax1 = ax1.smooth(method='SG', deg=1, win=21, doEdges=True, sigma=1)
    #     ax1 = ax1.show(
    #                     cmap_norm='equalize', 
    #                     contours=True, 
    #                     azdeg=45, altdeg=45, 
    #                     title='Residual 2.67', 
    #                     # zf=1000,
    #                     figsize=(10, 10)
    #                     )
        
    #     set_map(ax1, frame=True, dxdy=1000)
    
    # ax1.plot(df['X'], df['Y'], color='black', linestyle='None', marker='+', markersize=2, alpha=0.5)
    
    # for i in z:
    #     ax1 = grid1.up(z=i)
    #     # ax1 = ax1.dxdy(
    #     #                 method='SG', 
    #     #                 deg=3, 
    #     #                 win=5, 
    #     #                 doEdges=True, 
    #     #                 fs_tap=5
    #     #                 )
    #     ax1 = ax1.show(
    #                     cmap_norm='equalize', 
    #                     contours=True, 
    #                     azdeg=45, altdeg=45, 
    #                     title='Regional 2.67', 
    #                     # zf=1000,
    #                     figsize=(10, 10)
    #                     )
    #     set_map(ax1, frame=True, dxdy=1000)
    #     # ax1.save(file_output)

    # ax1.plot(df[X], df[Y], color='black', linestyle='None', marker='+', markersize=1, alpha=0.5)

    
    


In [110]:
data_file = '../../../data/data_gravity/satellite_gravity_jai.xlsx'
# data_file = '../../../data/data_gravity/2020_gravity_pth.xlsx'
file_output_prefix = '../map/patuha_grav_'

gen_data =[
    # 'data'
    'A',
    # 'PTH_A', 'PTH_B', 'PTH_C',
    # 'CAN_A', 'CAN_B', 'CAN_C',
    # 'CUT_A', 'CUT_B',
    # 'AWE_A', 'AWE_B',
    # 'WAE_A', 'WAE_B', 'WAE_C',
    # 'JAI_A', 'JAI_B'
]

for j in gen_data:
    df = pd.read_excel(data_file, sheet_name=j, header=0, na_values=None)

    gen_maps = [
        # 'Z', 'Gobs', 'FAA',
        # 'CBA 1.00', 'CBA 2.00', 'CBA 2.10', 'CBA 2.20', 'CBA 2.30', 'CBA 2.40', 'CBA 2.50', 'CBA 2.60', 'CBA 2.67'
        # 'SBA 1.00', 'SBA 2.00', 'SBA 2.10', 'SBA 2.20', 'SBA 2.30', 'SBA 2.40', 'SBA 2.50', 'SBA 2.60', 'SBA 2.67'
        # 'Z', 'Z_0', 'Z_1'
        # 'Z', 'Gravity Disturbances', 'Gravity Accelerations', 'NS DoVs', 'EW DoVs', 'SBA 2.67'
        # 'SBA_2.67'
        # 'Z', 'Gobs', 'FAA', 'SBA_2.67'
        # 'Z', 'Quasigeoid', 'Gobs', 'FAA', 'SBA_2.67'
        'Z'
        # 'CBA_2.00'
    ]

    for i in gen_maps:
        file_output = file_output_prefix + i + '.tiff'
        main(i, df, file_output, name=i)

CRSError: The WKT could not be parsed. OGR Error code 5